In [67]:
%pip install mpqp

Note: you may need to restart the kernel to use updated packages.


In [68]:
from mpqp import QCircuit
from mpqp.gates import H

def prepare_initial_state(n_precision_qubits, eigenstate_circuit):
    """
    Prepare the initial state for the Quantum Phase Estimation (QPE) algorithm.
    This function creates a quantum circuit that initializes the precision qubits
    to the state |+> and appends the provided eigenstate circuit to the bottom qubits.
    The total number of qubits in the circuit is the sum of the precision qubits
    and the qubits in the eigenstate circuit.

    :param eigenstate_circuit: The quantum circuit representing the eigenstate.
    :param n_precision_qubits: The number of precision qubits to be used in the QPE.
    :return: A QCircuit object representing the initial state for QPE.
    """

    # Create an empty circuit with total number of qubits
    n_eigenstate_qubits = eigenstate_circuit.nb_qubits
    total_qubits = n_precision_qubits + n_eigenstate_qubits
    circ = QCircuit(nb_qubits=total_qubits, label="QPE Circuit")

    # Define the precision qubits and the circuits qubits range indices
    precisions = range(n_precision_qubits)
    circuits = range(n_precision_qubits, total_qubits)

    # Apply Hadamard gates to precision qubits (index 0 to n-1)
    for i in range(n_precision_qubits):
        circ.add(H(i))

    # Append the given eigenstate circuit offset to the "bottom" qubits
    circ.append(eigenstate_circuit, qubits_offset=n_precision_qubits)

    # Return circuit in state |+>^n \otimes |eigenstate>
    return circ

In [69]:
from math import pi as PI
from mpqp.gates import CNOT, TOF, CP, S, S_dagger, CZ, X, Y, Z, T, Rz

def transform_to_controlled_gate(circ, gate, i, n, target):
    """
    Transform a single-target gate into its controlled version based on the precision qubit index.
    This function modifies the quantum circuit by adding the controlled version of the gate
    with the control qubit at position `i` and the target qubit at the specified
    target position (offset by `n`).

    :param circ: The quantum circuit to which the controlled gate will be added.
    :param gate: The single-target gate to be transformed into a controlled gate.
    :param i: The index of the precision qubit that acts as the control.
    :param n: The total number of precision qubits.
    :param target: The target qubit position (offset by `n`).
    :return: None
    """

    if isinstance(gate, CNOT): # CNOT -> TOF
        control = gate.controls[0] + n
        circ.add(TOF([i, control], target))
    elif isinstance(gate, X):
        circ.add(CNOT(i, target))
    elif isinstance(gate, S):
        circ.add(CP(PI/2, i, target))
    elif isinstance(gate, Y):
        circ.add(S(target))
        circ.add(CNOT(i, target))
        circ.add(S_dagger(target))
    elif isinstance(gate, Z):
        circ.add(CZ(i, target))
    elif isinstance(gate, T):
        circ.add(CP(PI/4, i, target))
    elif isinstance(gate, H):
        circ.add(Rz(PI, target))
        circ.add(Rz(PI/2, target))
        circ.add(CNOT(i, target))
        circ.add(Rz(-PI, target))
        circ.add(Rz(-PI/2, target))
    else:
        raise ValueError("Unknown single-target gate in controlled unitary.")

In [70]:
from mpqp import Barrier

def create_controlled_unitary(circ, unitary, n_precision_qubits):
    """
    Create controlled unitary operations for the Quantum Phase Estimation (QPE) algorithm.
    This function applies controlled unitary operations based on the provided unitary circuit
    and the number of precision qubits. The controlled unitary operations are applied
    in reverse order, starting from the highest precision qubit.
    The unitary circuit is transformed to its controlled version with the control qubit
    at the specified precision qubit position.

    :param circ: The quantum circuit to which the controlled unitary operations will be added.
    :param unitary: The quantum circuit representing the unitary operation to be controlled.
    :param n_precision_qubits: The number of precision qubits used in the QPE.
    :return: None
    """

    print("Applying controlled unitary operations.")
    n = n_precision_qubits
    precisions = range(n)

    # Loop through precision qubits in reverse order
    for i in precisions[::-1]:
        print(f"Applying controlled unitary for precision qubit {i}")
        # Apply controlled unitary operator 2^(n-i-1) times (1, 2, 4, ...)
        iterations = 2 ** (n - i - 1)
        print(f"Applying controlled unitary {iterations} times.")
        for k in range(iterations):
            print(f"\t{iterations-k} remaining")

            # Transform the unitary circuit to its controlled version with control qubit at position i
            for gate in unitary.gates:
                if len(gate.targets) == 1:
                    transform_to_controlled_gate(circ, gate, i, n, target=gate.targets[0] + n)
                else:
                    raise ValueError("Only single-target gates are supported in controlled unitary.")
        circ.add(Barrier())
    print("Controlled unitary operations applied.")

In [71]:
from mpqp.gates import SWAP, CRk
from math import floor

def build_inverse_qft(n_qubits):
    """
    Build the inverse Quantum Fourier Transform (QFT) circuit.
    This function constructs the inverse QFT circuit by adding SWAP gates for qubit reordering
    and applying controlled rotations and Hadamard gates in reverse order.
    The inverse QFT is essential for the Quantum Phase Estimation (QPE) algorithm to extract
    the phase information from the quantum state.

    :param n_qubits: The number of qubits in the circuit.
    :return: A QCircuit object representing the inverse QFT circuit over n_qubits.
    """

    qftc = QCircuit(n_qubits, label="Inverse QFT Circuit")

    # Add SWAP gates for qubit reordering
    qftc.add([SWAP(i, n_qubits - 1 - i) for i in range(int(floor(n_qubits / 2)))])

    # Apply controlled rotations and Hadamard gates in reverse order
    j = n_qubits - 1
    while j >= 0:
        qftc.add([CRk(i + 1 - j, i, j).inverse() for i in range(j + 1, n_qubits)])
        qftc.add(H(j))
        j -= 1

    return qftc


def append_inverse_qft(circ, n_precision_qubits):
    """
    Append the inverse Quantum Fourier Transform (QFT) to the circuit.
    This function adds the inverse QFT gates to the circuit, which is essential for
    the Quantum Phase Estimation (QPE) algorithm to extract the phase information from the
    quantum state.

    :param circ: The quantum circuit to which the inverse QFT will be appended.
    :param n: The number of precision qubits used in the QPE.
    :return: None
    """

    print("Appending inverse Quantum Fourier Transform (QFT).")

    # Append the inverse QFT to the circuit on the precision qubits only (no offset)
    circ.append(build_inverse_qft(n_precision_qubits), qubits_offset=0)
    circ.add(Barrier())

    print("Inverse QFT appended to the circuit.")

In [72]:
from mpqp.measures import BasisMeasure

def QPE(unitary, eigenstate_circuit, n_precision_qubits):
    """
    Quantum Phase Estimation (QPE) algorithm implementation.
    This function constructs a quantum circuit for the QPE algorithm, which estimates the phase
    of an eigenstate of a unitary operator. The circuit prepares an initial state, applies
    controlled unitary operations, applies the inverse Quantum Fourier Transform (QFT), and
    measures the precision qubits to obtain the estimated phase.

    :param unitary: The unitary operator whose eigenstate is being estimated.
    :param eigenstate_circuit: The circuit that prepares the eigenstate of the unitary
    :param n_precision_qubits: The number of precision qubits used in the QPE.
    :return: A QCircuit object representing the QPE circuit.
    """

    #### 1. Prepare the initial state |+>^n \otimes |eigenstate> ####
    n, m = n_precision_qubits, eigenstate_circuit.nb_qubits
    circ = prepare_initial_state(n_precision_qubits, eigenstate_circuit)

    #### 2. Apply controlled unitary operators ####
    create_controlled_unitary(circ, unitary, n_precision_qubits)

    #### 3. Apply inverse QFT to precision qubits ####
    append_inverse_qft(circ, n_precision_qubits)

    #### 4. Measure precision qubits ####
    circ.add(BasisMeasure(targets=list(range(n)), shots=1024))
    print("Measurement added to precision qubits.")

    # Add a barrier for clarity
    circ.add(Barrier())
    return circ

In [73]:
from mpqp import *
from mpqp.gates import *
from collections import defaultdict
from mpqp.execution import run, IBMDevice


def extract_phase_from_eigenstate(unitary, eigenstate_circuit, n_precision_qubits, device=IBMDevice.AER_SIMULATOR_STATEVECTOR):
    """
    Extract the phase from an eigenstate of a unitary operator using Quantum Phase Estimation (QPE).
    :param unitary: The unitary operator circuit whose eigenstate phase is to be estimated.
    :param eigenstate_circuit: The circuit that prepares the eigenstate of the unitary
    :param n_precision_qubits: The number of precision qubits used in the QPE.
    :param device: The quantum device to run the circuit on (default is AER simulator
    :return: The estimated phase as a float.
    """
    print("Starting Quantum Phase Estimation (QPE) to extract phase from eigenstate.")

    # Build & run the Quantum Phase Estimation (QPE) circuit
    qpe_circuit = QPE(unitary, eigenstate_circuit, n_precision_qubits)
    qpe_circuit.pretty_print()

    result = run(qpe_circuit, device)

    # Extracting the phase from the measurement results
    mask            = (1 << n_precision_qubits) - 1
    weight_by_k     = defaultdict(float)
    for sample in result.samples:
        k = sample.index & mask
        weight = getattr(sample, "probability", None)
        if weight is None:
            weight = sample.count
        weight_by_k[k] += weight

    # Picking the k with the largest total weight
    best_k = max(weight_by_k, key=weight_by_k.get)

    # Converting to phase ϕ̂ = k / 2^n
    phase_estimate = best_k / (1 << n_precision_qubits)

    print(f"Estimated phase: {phase_estimate}")
    return phase_estimate

In [74]:
def eigenstate_1():
    c = QCircuit(1)
    c.add(H(0))
    return c

def unitary_Z():
    c = QCircuit(1)
    c.add(H(0))
    return c

unitary = unitary_Z()
eigenstate = eigenstate_1()

# Expecting 0 or 0.5
phase = extract_phase_from_eigenstate(unitary, eigenstate, n_precision_qubits=3)

Starting Quantum Phase Estimation (QPE) to extract phase from eigenstate.
Applying controlled unitary operations.
Applying controlled unitary for precision qubit 2
Applying controlled unitary 1 times.
	1 remaining
Applying controlled unitary for precision qubit 1
Applying controlled unitary 2 times.
	2 remaining
	1 remaining
Applying controlled unitary for precision qubit 0
Applying controlled unitary 4 times.
	4 remaining
	3 remaining
	2 remaining
	1 remaining
Controlled unitary operations applied.
Appending inverse Quantum Fourier Transform (QFT).
Inverse QFT appended to the circuit.
Measurement added to precision qubits.
QCircuit QPE Circuit: Size (Qubits, Cbits) = (4, 3), Nb instructions = 52
     ┌───┐                                                ░          »
q_0: ┤ H ├────────────────────────────────────────────────░──────────»
     ├───┤                                                ░          »
q_1: ┤ H ├────────────────────────────────────────────────░──────────»
     ├───┤

In [75]:
unitary = QCircuit(1)
unitary.add(S(0))
eigenstate = QCircuit(1)
eigenstate.add(X(0))  # |1⟩
n_precision_qubits = 4

# Expecting 0.25
phase = extract_phase_from_eigenstate(unitary, eigenstate, n_precision_qubits)

Starting Quantum Phase Estimation (QPE) to extract phase from eigenstate.
Applying controlled unitary operations.
Applying controlled unitary for precision qubit 3
Applying controlled unitary 1 times.
	1 remaining
Applying controlled unitary for precision qubit 2
Applying controlled unitary 2 times.
	2 remaining
	1 remaining
Applying controlled unitary for precision qubit 1
Applying controlled unitary 4 times.
	4 remaining
	3 remaining
	2 remaining
	1 remaining
Applying controlled unitary for precision qubit 0
Applying controlled unitary 8 times.
	8 remaining
	7 remaining
	6 remaining
	5 remaining
	4 remaining
	3 remaining
	2 remaining
	1 remaining
Controlled unitary operations applied.
Appending inverse Quantum Fourier Transform (QFT).
Inverse QFT appended to the circuit.
Measurement added to precision qubits.
QCircuit QPE Circuit: Size (Qubits, Cbits) = (5, 4), Nb instructions = 39
     ┌───┐          ░                    ░                                     »
q_0: ┤ H ├──────────░─

In [76]:
unitary = QCircuit(1); unitary.add(T(0))
eigenstate = QCircuit(1); eigenstate.add(X(0))  # |1⟩
n_precision_qubits = 5

# Expecting 0.125
phase = extract_phase_from_eigenstate(unitary, eigenstate, n_precision_qubits)

Starting Quantum Phase Estimation (QPE) to extract phase from eigenstate.
Applying controlled unitary operations.
Applying controlled unitary for precision qubit 4
Applying controlled unitary 1 times.
	1 remaining
Applying controlled unitary for precision qubit 3
Applying controlled unitary 2 times.
	2 remaining
	1 remaining
Applying controlled unitary for precision qubit 2
Applying controlled unitary 4 times.
	4 remaining
	3 remaining
	2 remaining
	1 remaining
Applying controlled unitary for precision qubit 1
Applying controlled unitary 8 times.
	8 remaining
	7 remaining
	6 remaining
	5 remaining
	4 remaining
	3 remaining
	2 remaining
	1 remaining
Applying controlled unitary for precision qubit 0
Applying controlled unitary 16 times.
	16 remaining
	15 remaining
	14 remaining
	13 remaining
	12 remaining
	11 remaining
	10 remaining
	9 remaining
	8 remaining
	7 remaining
	6 remaining
	5 remaining
	4 remaining
	3 remaining
	2 remaining
	1 remaining
Controlled unitary operations applied.
A